# Spatially Informed Traveling Salesman Problem
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/17da8FuXsDfI9DCuaT7rbKKl2twoFYSlB#scrollTo=qO4f61z7Pc_A)  
Developed by Wanhee Kim(Phd student at UTK) / Revised Date : 04/03/2024


## Sequence of spatially informed TSP
1. Measure adjacency based on spatial information
2. Generate distance matrix
3. Analyze TSP

## Analysis

In [4]:
# Make Toy Data
from spatialtsp import is_far_enough, generate_clustered_points
points=generate_clustered_points(10)
points

,geometry
0,POINT (13.00000 17.00000)
1,POINT (26.00000 33.00000)
2,POINT (41.00000 42.00000)
3,POINT (18.00000 9.00000)
4,POINT (43.00000 32.00000)
5,POINT (6.00000 16.00000)
6,POINT (35.00000 31.00000)
7,POINT (10.00000 12.00000)
8,POINT (31.00000 37.00000)
9,POINT (7.00000 14.00000)


In [5]:
# Calculate Standard Distance Matrix
from spatialtsp import calculate_distance_matrix
distance_matrix=calculate_distance_matrix(points)
distance_matrix

array([[   0, 2062, 3754,  943, 3354,  707, 2608,  583, 2691,  671],
       [2062,    0, 1749, 2530, 1703, 2625,  922, 2640,  640, 2687],
       [3754, 1749,    0, 4022, 1020, 4360, 1253, 4314, 1118, 4405],
       [ 943, 2530, 4022,    0, 3397, 1389, 2780,  854, 3087, 1208],
       [3354, 1703, 1020, 3397,    0, 4031,  806, 3859, 1300, 4025],
       [ 707, 2625, 4360, 1389, 4031,    0, 3265,  566, 3265,  224],
       [2608,  922, 1253, 2780,  806, 3265,    0, 3140,  721, 3276],
       [ 583, 2640, 4314,  854, 3859,  566, 3140,    0, 3265,  361],
       [2691,  640, 1118, 3087, 1300, 3265,  721, 3265,    0, 3324],
       [ 671, 2687, 4405, 1208, 4025,  224, 3276,  361, 3324,    0]])

In [8]:
# set working directory
# import os

# path = 'D:/GIS_analyzing/1.Standard_TSP/0.test_iteration' # write your own directory
# os.chdir(path)
# os.getcwd()

## Display Basemap

In [1]:
## Add basemap
from spatialtsp import Map

my_map = Map(center=[40.7128, -74.0060], zoom=10)
my_map.add_basemap("OpenStreetMap.Mapnik")

my_map


Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [2]:
## Add GeoJSON
from spatialtsp import Map

geojson_map = Map(center=[37.0902, -95.7129], zoom=4)
geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_states.geojson"
geojson_map.add_geojson(geojson_url, name="US Counties")

geojson_map


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [3]:
## Add shp
import geopandas as gpd
import requests
import zipfile
import io
from spatialtsp import Map

# Shapefile URL
url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Download and extract the zip file
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Find the shapefile name
shp_name = [name for name in z.namelist() if '.shp' in name][0]

# Read and extract the shapefile
z.extractall("temp_shp")
states_gdf = gpd.read_file("temp_shp/us_states.shp")


# Display the shapefile
my_map = Map(center=[37.0902, -95.7129], zoom=4)
my_map.add_vector(states_gdf, name="US States")

my_map


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [4]:
## Add Vector
from spatialtsp import Map

# Initialize the map
my_map = Map(center=[37.0902, -95.7129], zoom=4)

# GeoJSON URLs
city_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_cities.geojson"
county_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"

# Shapefile URL
states_shape_url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Load GeoJSONs as GeoDataFrames
gdf_cities = gpd.read_file(city_geojson_url)
gdf_counties = gpd.read_file(county_geojson_url)

# Use add_vector to add GeoDataFrames to the map
my_map.add_vector(gdf_counties, name="US Counties")
my_map.add_vector(gdf_cities, name="US Cities")
# For the shapefile, directly use the URL
my_map.add_vector(states_shape_url, name="US States")

# Display the map
my_map

Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [ ]:
## other codes will be updated soon